## Python 계산 모델

1. `리스트`는 파이썬의 배열 구현이다. 따라서 Random Access Machine이다.
2. `객체`가 상수 개의 속성을 지닌다면 이는 Pointer Machine이다.

### Python 계산 비용

1. list

   파이썬의 배열 구현임

   - append : O(1)

   - concat : O(1 + n+m)

     L1과  L2를 합쳐 새로운 리스트로 만드는 걔념이다.

   - extend : O(m)

     L1의 뒤에 L2를 붙이는 개념이다

   - slice :  O(j - i + 1)

   - has : O(n)

   - len : O(1)

   - sort : O(nlogn)

2. tuple, str

   Immutable한 list로 보면 된다.

3. dict

   파이썬의 해시테이블 구현이다.

4. set

   val 없는 dict와 같다

5. heapq

   STL에 구현된 힙이다.

   - push : O(logn)
   - pop : O(logn)

6. long

   크기가 큰 정수자료이다.

   - x + y = O(|x| + |y|)
   - x * y = O((|x| + |y|)^lg3)

## 예제 : 문서 거리 계산하기

#### 목적 : 두 문서의 거리 $d(D_1, D_2)$를 계산하라.

#### 응용 : 위키에서 중복 문서를 발견. 쿼리 $D_2$에 대응하는 문서 검색.

#### 개념

- word는 알파벳의 연속이다 = list of character
- document는 word의 연속이다 = list of word
- distance는 공통된 word가 많을 수록 짧아진다.
- 문서 D에서 D[w]는 w가 등장하는 빈도이다 = 문서 D는 단어 빈도 벡터

#### 해결 방법

- 두 벡터의 방향이 비슷하면 사이각이 작다

- scale invariant 하도록 정규화한다.
  $$
  d_{1,2} = \arccos( {{D_1 \cdot D_2} \over {|D_1||D_2|}})
  $$

#### 알고리즘

단어 빈도 벡터 D를 만드는 것이 핵심. 

1. 문서를 단어 별로 쪼갠다.

   a. 정규표현식을 사용할까? O(exp)

   b. char를 훑으면서 !isalphanum 마다 쪼갤까? O(|doc|)

2. 각 단어의 빈도를 센다.

   a. 애초에 딕셔너리를 사용하거나

   b. word 리스트를 만들어 **정렬**하고, 이전 word와 같을 카운터 증가. 다를 경우 이전 word와 카운터를 자료구조에 저장한다.

3. 내적 값을 구한다.

   0이 아닐 경우에만 연산시켜라

   a. 문서 A의 한 word마다 상대 문서에서 word가 있는지 검사하여 내적

   b. **정렬된 상태를 이용**하여, 양측 워드 리스트의 지시자를 옮기면서 내적

In [58]:
import math
import sys

In [59]:
def wordvec_from_file(filename) :
    L = get_filelines(filename)
    wordvec = {}
    for line in L :
        wordvec_from_string(wordvec, line)
    return wordvec

def get_filelines(filename) :
    try :
        f = open(filename, 'r')
        return f.readlines();
    except IOError :
        print("Error has been occured opening file :", filename)
        sys.exit();
        
def wordvec_from_string(wordvec, line) :
    word = ""
    for c in line :
        if c.isalnum() :
            word += c
        elif len(word) > 0 :
            word = word.lower()
            increase_word_counter(wordvec, word)
            word = ""
    if len(word) > 0 :
        word = word.lower()
        increase_word_counter(wordvec, word)
    
def increase_word_counter(wordvec, word) :
    try :
        wordvec[word] += 1
    except :
        wordvec[word] = 1

In [60]:
D1 = wordvec_from_file("./data/t1.verne.txt")
D2 = wordvec_from_file("./data/t2.bobsey.txt")
D3 = wordvec_from_file("./data/t3.lewis.txt")

In [61]:
def wordvec_innerproduct(D1, D2) :
    summation = 0
    for word in D1.keys() :
        if word in D2.keys() :
            summation += D1[word] * D2[word]
    return summation

def wordvec_distance(D1, D2) :
    denorm = wordvec_innerproduct(D1, D2)
    norm = denorm / math.sqrt(wordvec_innerproduct(D1, D1) * wordvec_innerproduct(D2, D2))
    return math.acos(norm)

In [62]:
wordvec_distance(D1, D1)

0.0

In [63]:
wordvec_distance(D1, D2)

0.5829494425772849

In [64]:
wordvec_distance(D1, D3)

0.42184872505837584

In [65]:
wordvec_distance(D2, D2)

0.0

In [66]:
wordvec_distance(D2, D3)

0.5741596010646867

In [68]:
wordvec_distance(D3, D3)

0.0

In [69]:
same0 = wordvec_from_file("./data/same0")
same1 = wordvec_from_file("./data/same1")
wordvec_distance(same0, same1)

0.0